In [ ]:
import os

os.chdir('/PINNs')

cuda_device = '3'
os.environ["CUDA_VISIBLE_DEVICES"] = cuda_device
# os.environ['CUBLAS_WORKSPACE_CONFIG']=':4096:8'

import math

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import traceback
from collections import OrderedDict
from tqdm import tqdm, trange

import random

from tensorboardX import SummaryWriter

from scipy import integrate

sns.set_style("white")
torch.cuda.is_available()


from PINNs import parallel_train
from parameters import params


torch.set_default_dtype(torch.float64)
torch.set_default_device("cuda")


# Parallel training

In [ ]:
import torch.multiprocessing as mp

num_processes = 5

width_R = 4

param_vector = [4, 8, 16, 32, 64]

param_vector = [width_R + i for i in param_vector]

print(param_vector)

result_matrix = np.zeros((len(param_vector), num_processes, 4))


i = 0

for width in tqdm(param_vector):
    
    shared_params = params()

    shared_params.width = width
    shared_params.h_train = 2/256
    shared_params.num_hidden = 2
    shared_params.epoches = 30000

    if __name__ == "__main__":
            
        mp.set_start_method('spawn', force=True)
        
        manager = mp.Manager()
        results_queue = manager.Queue()

        mp.spawn(parallel_train, args=(shared_params, results_queue), nprocs=num_processes, join=True)
        
        # print(results_queue.get())

        while not results_queue.empty():
            (idx, loss_vector) = results_queue.get()
    #             print(f"Index: {idx}, error: {error}, error_infty: {error_infty}")
            result_matrix[i, idx, :] = loss_vector
            
    i += 1

print(result_matrix)